In [2]:
import langchain


In [3]:
from langchain.agents import initialize_agent, Tool

from dotenv import load_dotenv
import os


load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

In [4]:
# !pip install -U duckduckgo-search
from langchain_google_genai import GoogleGenerativeAI

from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

# Web Search Tool using DuckDuckGo
search_tool = Tool(
    name="DuckDuckGo Search",
    func=DuckDuckGoSearchRun().run,
    description="Use this tool to perform web searches using DuckDuckGo."
)



from langchain_groq import ChatGroq

llm = ChatGroq(model_name="llama-3.3-70b-versatile", groq_api_key=groq_api_key) 
tools = [search_tool]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

response = agent.run("What are the top trending movies this week in india ?")

print(response)

/var/folders/fd/dk1cn0wn0j3b7_0hf6vgbgp40000gn/T/ipykernel_57906/853456106.py:20: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
/var/folders/fd/dk1cn0wn0j3b7_0hf6vgbgp40000gn/T/ipykernel_57906/853456106.py:22: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed 



> Entering new AgentExecutor chain...
To find the top trending movies this week in India, I should first look for a reliable source that provides up-to-date information on movie trends. 

Action: DuckDuckGo Search
Action Input: top trending movies in India this week
Observation: 2025 is kicking off with a bang, and the excitement is real! From January 13 to January 19, a range of new release movies in theaters this week in Hindi, English, Tamil, Telugu, and more. Other OTT releases this week in India. In addition, a few more series and movies were released this week. Here's a quick look at them: ... Trending News. iQOO Neo 10R Moonknight Titanium colour, AnTuTu score, and pricing teased ahead of launch ... Top 20 South Indian movies 2023 on OTT: 2018, Ponniyin Selvan 2, and more. December 28, 2023 ... Yami Gautam and Pratik Gandhi's Action-Packed movie Dhoom Dhaam is all set to premiere on Netflix from 14th February 2025. The story of Dhoom Dhaam revolves around Veer Khurana and Koya

In [5]:
# The WikipediaAPIWrapper - allows you to query Wikipedia for articles and summaries.

#!pip install wikipedia

from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

# Step 1: Initialize the Wikipedia Tool
wikipedia_tool = Tool(
    name="Wikipedia Search",
    func=WikipediaAPIWrapper().run,
    description="Use this tool to search Wikipedia for information on any topic."
)

# llm = GoogleGenerativeAI(model="gemini-pro", temperature=0.5)

llm = ChatGroq(model_name="llama-3.3-70b-versatile", groq_api_key=groq_api_key) 

tools = [wikipedia_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

query = "Who is Narendra Modhi?"
response = agent.run(query)
print("Response from Agent:")
print(response)



> Entering new AgentExecutor chain...
To find information on Narendra Modhi, I should first search for his name on Wikipedia to gather details about his life, achievements, and political career.

Action: Wikipedia Search
Action Input: Narendra Modhi
Observation: No good Wikipedia Search Result was found
Thought:Since the initial search for "Narendra Modhi" did not yield a satisfactory result, I should try a different spelling, as it's possible that the name is misspelled. The correct spelling is likely "Narendra Modi", who is a well-known political figure.

Action: Wikipedia Search
Action Input: Narendra Modi
Observation: Page: Narendra Modi
Summary: Narendra Damodardas Modi (born 17 September 1950) is an Indian politician who has served as the prime minister of India since 2014. Modi was the chief minister of Gujarat from 2001 to 2014 and is the member of parliament (MP) for Varanasi. He is a member of the Bharatiya Janata Party (BJP) and of the Rashtriya Swayamsevak Sangh (RSS), a 

In [6]:
import sqlite3

# Connect to SQLite database (creates example.db if it doesn't exist)
connection = sqlite3.connect("example.db")
cursor = connection.cursor()

# Create the inventory table
cursor.execute("""
CREATE TABLE IF NOT EXISTS inventory (
    id INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    stock_quantity INTEGER NOT NULL
)
""")

# Insert sample data
cursor.executemany("""
INSERT INTO inventory (product_name, stock_quantity) VALUES (?, ?)
""", [
    ("Product A", 150),
    ("Product B", 200),
    ("Product C", 300),
    ("Product D", 100),
    ("Product E", 50)
])

# Commit and close connection
connection.commit()
connection.close()

print("Database created and populated with sample data.")


Database created and populated with sample data.


In [7]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

In [8]:
# Step 1: Connect to the Database
db = SQLDatabase.from_uri("sqlite:///example.db")

# Step 3: Initialize the LLM
# llm = GoogleGenerativeAI(model="gemini-pro", temperature=0.5)
llm = ChatGroq(model_name="llama-3.3-70b-versatile", groq_api_key=groq_api_key) 


# Step 2: Create the SQL Database Toolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)


query1 = "How many units of Product A are in stock?"
response1 = agent.run(query1)
print("Response:", response1)



> Entering new AgentExecutor chain...
To determine how many units of Product A are in stock, I first need to identify which tables in the database might contain inventory or product information. 

Action: sql_db_list_tables
Action Input: 
Observation: inventory
Thought:With the table name "inventory" from the observation, I can now query the schema of this table to understand its fields and determine which field might contain the product name and the quantity in stock.

Action: sql_db_schema
Action Input: inventory
Observation: 
CREATE TABLE inventory (
	id INTEGER, 
	product_name TEXT NOT NULL, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from inventory table:
id	product_name	stock_quantity
1	Product A	150
2	Product B	200
3	Product C	300
*/
Thought:With the schema of the "inventory" table, I now understand that the "product_name" field contains the name of the product and the "stock_quantity" field contains the quantity in stock. To find out how many units of P

In [9]:
# ArxivQueryRun
from langchain.agents import initialize_agent, Tool

from dotenv import load_dotenv
import os

load_dotenv()
# google_api_key = os.getenv("GOOGLE_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

from langchain_community.tools.arxiv.tool import ArxivQueryRun

arxiv_tool = Tool(
    name="Arxiv Search",
    func=ArxivQueryRun().run,
    description="Use this tool to search academic papers on arXiv. Provide topics, authors, or keywords."
)

# Step 2: Initialize the LLM
# llm = GoogleGenerativeAI(model="gemini-pro", temperature=0.5)
llm = ChatGroq(model_name="llama-3.3-70b-versatile", groq_api_key=groq_api_key) 

# Step 3: Combine the tool into a list
tools = [arxiv_tool]

# Step 4: Initialize the Agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

# Step 5: Query the Agent
query = "Search for research papers on Diffusion Models in 2024."
response = agent.run(query)
print("Response from Agent:")
print(response)




> Entering new AgentExecutor chain...
To find research papers on Diffusion Models in 2024, I should utilize the Arxiv Search tool, focusing on the specific topic of "Diffusion Models" and limiting the search to papers published in 2024.

Action: Arxiv Search
Action Input: {"query": "Diffusion Models", "date": "2024"}
Observation: No good Arxiv Result was found
Thought:Since the initial search didn't yield any relevant results, I should try to refine my search query to better capture the topic of interest. Perhaps using more specific keywords related to Diffusion Models, such as "Deep Learning" or "Generative Models", could help improve the search results.

Action: Arxiv Search
Action Input: {"query": "Diffusion Models AND Deep Learning", "date": "2024", "keywords": "Generative Models"}
Observation: No good Arxiv Result was found
Thought:Given that the previous searches did not yield satisfactory results, I should consider expanding the search criteria to include a broader range of to

In [10]:
## With Groq APi KEy

In [1]:
# Import necessary libraries
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.tools import Tool
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")  # Corrected variable name

# Import Arxiv Search Tool
from langchain_community.tools.arxiv.tool import ArxivQueryRun

arxiv_tool = Tool(
    name="Arxiv Search",
    func=ArxivQueryRun().run,
    description="Use this tool to search academic papers on arXiv. Provide topics, authors, or keywords."
)

# Step 2: Initialize the LLM
from langchain_groq import ChatGroq

llm = ChatGroq(model_name="llama3-8b-8192", groq_api_key=groq_api_key)  # Corrected API key usage

# Step 3: Combine the tool into a list
tools = [arxiv_tool]

# Step 4: Define a Better System Prompt
system_message = """
You are an AI assistant helping with academic research. 
When using tools, always follow the ReAct format:
1. Thought: Explain your reasoning.
2. Action: Name of the tool (e.g., "Arxiv Search").
3. Action Input: Provide a valid input for the tool.

Example:
Thought: I need to search for recent research on Diffusion Models in 2024.
Action: Arxiv Search
Action Input: "Diffusion Models 2024"

If no tool is needed, simply return the answer directly.
"""

# Step 5: Initialize the Agent with Custom Prompt
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Corrected agent type
    verbose=True,
    agent_kwargs={"system_message": system_message}  # Forces correct format
)

# Step 6: Query the Agent
query = "Search for research papers on Diffusion Models in 2024."
response = agent.run(query)

# Print the response
print("Response from Agent:")
print(response)


/var/folders/fd/dk1cn0wn0j3b7_0hf6vgbgp40000gn/T/ipykernel_58020/3635848580.py:45: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
/var/folders/fd/dk1cn0wn0j3b7_0hf6vgbgp40000gn/T/ipykernel_58020/3635848580.py:55: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(



> Entering new AgentExecutor chain...
Let's get started!

Thought: I need to find research papers on Diffusion Models published in 2024. I can use the Arxiv Search tool to search for papers.

Action: Arxiv Search

Action Input: "diffusion models" AND "2024"

Observation: No good Arxiv Result was found
Thought:Thought: It seems like the search query didn't yield any results. Let me try to refine the search query.

Action: Arxiv Search

Action Input: ("diffusion models" OR "diffusion-based models" OR "diffusion-based methods" OR "diffusion-based techniques") AND ("2024" OR "2024.01" OR "2024.02" OR "2024.03" OR "2024.04" OR "2024.05" OR "2024.06" OR "2024.07" OR "2024.08" OR "2024.09" OR "2024.10" OR "2024.11" OR "2024.12")

Observation: No good Arxiv Result was found
Thought:Thought: Hmm, it seems like the search query still didn't yield any results. I'll try to think of other ways to refine the search query.

Action: Arxiv Search

Action Input: ("diffusion models" OR "diffusion-based

In [9]:
from langchain_community.tools.file_management import ReadFileTool
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

from dotenv import load_dotenv
import os

load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
# Step 1: Initialize the ReadFileTool
read_file_tool = Tool(
    name="Read File",
    func=ReadFileTool().run,
    description="Use this tool to read the content of a text file in the current working directory. Provide the file name as input."
)

# Step 2: Initialize the LLM
llm = GoogleGenerativeAI(model="gemini-pro", temperature=0.5)

# Step 3: Combine the tool into a list
tools = [read_file_tool]

# Step 4: Initialize the Agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

# Step 5: Query the Agent with a file name
file_name = "example.txt"

response = agent.run(f"Read the content of the file named {file_name}.")
print("Response from Agent:")
print(response)




> Entering new AgentExecutor chain...
Action: Read File
Action Input: example.txt
Observation: Hy Welcome to my Txt File
Thought:

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


Final Answer: Hy Welcome to my Txt File

> Finished chain.
Response from Agent:
Hy Welcome to my Txt File
